In [604]:
import os
import re
import time
import pickle
import pprint
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from collections import Counter
from bs4 import BeautifulSoup as bs
from get_credentials import get_creds
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (NoSuchElementException,
                                        ElementNotInteractableException,
                                        ElementClickInterceptedException) 

if os.path.exists('credentials.pickle'):
    with open('credentials.pickle','rb') as f:  
        credentials = pickle.load(f)
else:
    get_creds()
    with open('credentials.pickle','rb') as f:  
        credentials = pickle.load(f)

TEAMS_PATH = r'https://login.microsoftonline.com/common/oauth2/authorize?response_type=id_token&client_id=5e3ce6c0-2b1f-4285-8d4b-75ee78787346&redirect_uri=https%3A%2F%2Fteams.microsoft.com%2Fgo&state=00b8dd1b-8f5d-4eb3-8f77-5c76ff40e3b2&&client-request-id=d2afe301-0371-4fb6-b185-1ad0345268b5&x-client-SKU=Js&x-client-Ver=1.0.9&nonce=3afa9b8e-d754-4aae-bb73-ab6fd6bc2f33&domain_hint='
EXECUTABLE_PATH = credentials['EXECUTABLE_PATH']
EMAIL_ID = credentials['Email_or_phone_no']
PASSWORD = credentials['Password']
TEAM = "DRISHTI-ML"

status = [
    'INSTALLATION IN PROGRESS',
    'INSTALLATION DONE',
    
    'ASSIGNMENT1 IN PROGRESS',
    'ASSIGNMENT1 DONE',
    
    'ASSIGNMENT2 IN PROGRESS',
    'ASSIGNMENT2 DONE',
    
    'ASSIGNMENT3 IN PROGRESS',
    'ASSIGNMENT3 DONE',
    
    'ASSIGNMENT4 IN PROGRESS',
    'ASSIGNMENT4 DONE',         
]
for i,ele in enumerate(status):
    print(i,"-->",ele)

0 --> INSTALLATION IN PROGRESS
1 --> INSTALLATION DONE
2 --> ASSIGNMENT1 IN PROGRESS
3 --> ASSIGNMENT1 DONE
4 --> ASSIGNMENT2 IN PROGRESS
5 --> ASSIGNMENT2 DONE
6 --> ASSIGNMENT3 IN PROGRESS
7 --> ASSIGNMENT3 DONE
8 --> ASSIGNMENT4 IN PROGRESS
9 --> ASSIGNMENT4 DONE


In [605]:
#cke_348_contents > div:nth-child(1) 

In [623]:
def get_bot(refresh = True,bot = 1):
    if bot == 1: #main bot
        BOT = 'https://teams.microsoft.com/_#/school/conversations/BOT%20(beta)?threadId=19:72818b8c80fb4045bf22a2f19e8af04b@thread.tacv2&ctx=channel'
    else: #private bot
        print('getting private bot')
        BOT = 'https://teams.microsoft.com/_#/school/conversations/BOT?threadId=19:84124691e65d4ebabaee9d78e41ee04f@thread.tacv2&ctx=channel'
    time.sleep(2)
    driver.get(BOT)
    time.sleep(2)
    if refresh:
        driver.refresh()
        time.sleep(5)

def get_message_stuff():
    #get all message_bodies
    message_bodies = driver.find_elements_by_id('messageBody')
    #get page_source and findall message_ids
    message_ids = re.findall(r'<div id="(m[0-9]*)"',str(driver.page_source))
    return message_bodies,message_ids



#grab the message
def edit(message_bodies,message_ids,bot = 1):
    try:
        message_to_edit = message_bodies[INDEX_OF_MESSAGE]
        driver.execute_script("return arguments[0].scrollIntoView(true);", message_to_edit)
        driver.find_element_by_xpath("//body").click() #click somewhere else on the body
        time.sleep(0.5)
        hover = ActionChains(driver).move_to_element(message_to_edit)
        hover.perform()
        time.sleep(0.5)
        driver.find_element_by_css_selector('.message-actions-more').click()#click on "..."
        time.sleep(0.5)
        driver.find_element_by_css_selector("li.message-action-overflow-menu-item:nth-child(2) > button:nth-child(1) > span:nth-child(2)").click()#select edit        
    except:
        get_bot(refresh = True,bot = bot)
        message_bodies,message_ids = get_message_stuff()
        message_bodies,message_ids = edit(message_bodies,message_ids)
    return message_bodies,message_ids
        
    
def generate_string(students):
    student_info = ''
    student_list = [[x]+students[x] for x in students]
    student_list = sorted(student_list,key=lambda student_list: int(student_list[2]), reverse=True)
    for i,s in reversed(list(enumerate(status))):
        sub_list = [x[:-1] for x in student_list if x[-1]==f'{str(i)}']
        if not sub_list==[]:
            student_info+="\n\n>>> "+s+"\n\n"
        student_info += '\n'.join([' '.join(ele) for ele in sub_list]) 
    return student_info
    

def new_student_entry():
    print('trying to get new message metadata')
    message_bodies,message_ids = get_message_stuff()
    
    name,uid,cur_status = None,None,None
    
    try:
        for k in range(-1,-900,-1):
            try:
                inner_html = driver.find_element_by_id(message_ids[k]).get_attribute('innerHTML')
                if k==-1:
                    cur_status = re.findall('<div>([0-9])</div>',inner_html)[0]
                (name,uid)= re.findall('alt="Profile picture of ([A-Z ]*)." upn="([0-9A-Z]*)@svnitsuratg.onmicrosoft.com"',inner_html)[0]
                name = name.split()[-1]
                break
            except IndexError:
                continue
    except :
        print('failed')
        time.sleep(0.1)
        new_student_entry()
    print('success',name,uid,cur_status)
   
    assert  name != None
    assert  uid != None
    assert  cur_status != None
    return name,uid,cur_status

def refresh_status():
    students = update_in_code()
    print('clicking on edit')
    message_bodies,message_ids = edit(message_bodies,message_ids)
    print('dumping payload')
    dump_payload(students)    

def remove(students,uid):
    del students[uid]
    update_local()
    return students
    
def update_local():
    with open('students.pickle','wb') as f:
        pickle.dump(students,f)

def update_in_code():
    with open('students.pickle','rb') as f:
        students = pickle.load(f)
    return students

def send_reply(payload):
    #click on reply button
    driver.find_elements_by_class_name('thread-action-reply-message')[-1].click()
    in_htlm = str(driver.find_element_by_class_name('ts-edit-box').get_attribute("innerHTML"))
    cke_element = re.findall(r'<div id="(cke_[0-9]*_contents)"',in_htlm)[0]
    
    #load data into clipboard
    df = pd.DataFrame([payload])
    df.to_clipboard(index=False,header=False)
    
    #clear prev text
    cke_element.send_keys(Keys.CONTROL, "a")
    cke_element.send_keys(Keys.DELETE)

    time.sleep(0.5)
    
    #dump payload
    cke_element.send_keys(Keys.CONTROL, "v")
    cke_element.send_keys(Keys.RETURN)
#     print('success!')


def reply(command,*args):
    
    def add_prefix(string):
        string = "BOT=>\n"+string
        return string
    
    if command == "STATS":
        departments = [re.split('U19([A-Z][A-Z])',key)[1] for key in students]
        percentages = list(map(lambda x: (x/len(students))*100,list(Counter(departments).values())))
        percentages = [str(x) for x in list(np.round(percentages,3))]
        departments = list(Counter(departments).keys())
        stats = '\n'.join([departments[i]+' --> '+percentages[i]+'%' for i in range(0,len(departments)) ])
        stats = f"Active Students: {len(students)}\nDepartment wise:\n"+stats
        payload = add_prefix(stats)
        send_reply(payload)
        
    elif command == "STOP":
        send_reply(add_prefix("BOT SHUTTING DOWN. GOODBYE"))
        
    elif command == "SET_CHECK_PERIOD":
        send_reply(add_prefix("CHECK_PERIOD =="+sleep_time))
        
    elif command == "REMOVE":
        send_reply(add_prefix("REMOVED "+uid))
        
    elif command == "REFRESH":
        send_reply(add_prefix("REFRESHED"))
        
    elif command == "SWITCH BOT":
        send_reply(add_prefix("SWITCHING TO PRIVATE BOT"))
        
    else: #incorrect command
        send_reply(add_prefix("INVALID COMMAND"))

    
def detect_change(num_of_messages,sleep_time=1):
    while True:
        message_bodies,message_ids = get_message_stuff() 
        command=None
        
        #check if num_of_messages is greater that what it was
        if num_of_messages < len(message_ids):
            num_of_messages = len(message_ids)       
            print('change detected')
            time.sleep(0.2)
            
            #get student data
            name,uid,cur_status = new_student_entry()
            
            #check if admin, and if ADMIN_COMMAND
            if name=="Aditya" and uid=="U18EC013" and cur_status.split(' ')[0]=="ADMIN_COMMAND":
                cur_status_split = cur_status.split(' ')
                if len(cur_status_split)== 2:
                    command = cur_status_split[-1]
                elif len(cur_status_split)== 3:
                    command = cur_status_split[-2]
                    
                if command=="STATS":
                    reply(command)
                    
                elif command=="STOP":
                    reply(command)
                    print('STOP command triggered. exiting...')
                    break
                    
                elif command=="SET_CHECK_PERIOD":
                    sleep_time = int(cur_status_split[-1])
                    reply(command,sleep_time)
                    
                elif command=="REMOVE":
                    uid = int(cur_status_split[-1])
                    students = remove(uid)
                    reply(command,uid)
                    
                elif command=="REFRESH":
                    refresh_status()
                    reply(command)
                    
                elif command=="SWITCH BOT":
                    reply(command)
                    break
                    
                else:
                    reply(command)
                        
            #check if student status is fresh and is a number
            elif (students[uid]!= [name,cur_status]) and (cur_status in str(list(range(0,len(status))))):
                print('status valid')
                
                #assign students[uid] to fresh values
                #and save students locally
                students[uid]=[name,cur_status]
                
                update_local()                
                print('clicking on edit')
                message_bodies,message_ids = edit(message_bodies,message_ids)
                print('dumping payload')
                dump_payload(students)
        else:
            print('waiting for message...')
            time.sleep(sleep_time)
            
    if command=="SWITCH BOT":
        get_bot(refresh = False,bot = -1) #get private bot
        detect_changes()

def dump_payload(students):
    #get payload
    payload = generate_string(students)
    
    #find cke_element to dump payload into
    message_ids = re.findall(r'<div id="(m[0-9]*)"',str(driver.page_source))
    message_id_outer_htlm = str(driver.find_element_by_id(message_ids[INDEX_OF_MESSAGE]).get_attribute('outerHTML'))
    cke_contents_id_name = re.findall(r'<div id="(cke_[0-9]*_contents)"',message_id_outer_htlm)[0]
    cke_wysiwyg_div  = f"#{cke_contents_id_name} > div:nth-child(1)"
    cke_element = driver.find_element_by_css_selector(cke_wysiwyg_div)
    
    #load data into clipboard
    df = pd.DataFrame([payload])
    df.to_clipboard(index=False,header=False)
    
    #clear prev text
    cke_element.send_keys(Keys.CONTROL, "a")
    cke_element.send_keys(Keys.DELETE)

    time.sleep(0.5)
    
    #dump payload
    cke_element.send_keys(Keys.CONTROL, "v")
    cke_element.send_keys(Keys.RETURN)
    print('success!')

def main(bot=1,NEW_SESSION=False):
    #start get teams login
    driver.get(TEAMS_PATH)
    time.sleep(1)
    if NEW_SESSION:
        try:
            #try adding email
            email = driver.find_element_by_xpath('//*[@id="i0116"]')
            email.send_keys(EMAIL_ID) 
            email.send_keys(Keys.RETURN)
        except NoSuchElementException:
            #if not found then find and click stored email
            driver.find_element_by_xpath('/html/body/div/form[1]/div/div/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div/div[1]/div/div/div/div[2]/div').click()

        time.sleep(1)

        #enter password
        password = driver.find_element_by_xpath('//*[@id="i0118"]')
        password.send_keys(PASSWORD) 
        time.sleep(0.1)
        password.send_keys(Keys.RETURN)

        time.sleep(2)

        #hit yes
        driver.find_element_by_xpath('//*[@id="idSIButton9"]').click()

        time.sleep(5)

    get_bot(refresh=False,bot=bot)
    
    INDEX_OF_MESSAGE = 1
    
    message_bodies,message_ids = get_message_stuff()
    num_of_messages = len(message_ids)
    
    detect_change(num_of_messages,sleep_time=20)

In [624]:
BOT = -1
NEW_SESSION = False

In [626]:
if __name__ == "__main__":
    if NEW_SESSION:
        driver = webdriver.Firefox(executable_path=EXECUTABLE_PATH)
    main(BOT,NEW_SESSION)

getting private bot
waiting for message...
waiting for message...
waiting for message...
change detected
trying to get new message metadata
success ADITYA U18EC013 None


AssertionError: 

In [ ]:
'>\n\n\nADMIN_COMMAND STOP\n\n, reply from'

In [694]:
# inner_html

In [683]:
(1/(0.69*100*0.1))*1000

144.92753623188406

In [ ]:
">\n\n\nADMIN_COMMAND STOP\n\n, reply from"

In [699]:
# inner_html

In [702]:
soup = bs(inner_html,'html.parser')

In [731]:
search_string = soup.findAll("div", {"class": "screen-reader-text"})
texts = [a.text for a in search_string][0],replace('\n','')

In [732]:
texts

'\n\n\nADMIN_COMMAND STOP\n\n, reply from YATIN ADITYA TEKUMALLA ADITYA.'

In [706]:
soup.find('class','screen-reader-text')

In [700]:
# message_bodies,message_ids = get_message_stuff()

# inner_html = driver.find_element_by_id(message_ids[-1]).get_attribute('innerHTML')
# re.findall(">[^A-Z 0-9_]*([A-Z 0-9_]*).*, reply from",inner_html,re.DOTALL)

In [689]:
re.findall("^>\s+([A-Z]\w+ \w+)\s+, reply from",inner_html,re.DOTALL)

[]

In [674]:
re.match(">, reply from",inner_html,re.DOTALL)

In [458]:
# with open('students.pickle','rb') as f:
#     students = pickle.load(f)

In [457]:
# with open('students.pickle','wb') as f:
#     pickle.dump(students,f)

In [533]:
## useless comments and stuff... remove later


# message_bodies,message_ids = get_message_stuff()

# inner_html = driver.find_element_by_id(message_ids[-2]).get_attribute('innerHTML')
# inner_html

# message_ids[-2]

# outerHTML = driver.find_element_by_id(message_ids[-2]).get_attribute('outerHTML')
# outerHTML

# names =['GAURAV U19EC029     ASSIGNMENT2 IN PROGRESS',
#  'SHASHWATHA U19CS105     INSTALLATION DONE',
#  'PRABHAKAR U19EE073     ASSIGNMENT2 DONE',
#  'PEEPAM U19EE060     INSTALLATION DONE',
#  'ATUL U19ME108     INSTALLATION DONE',
#  'PRAKHAR U19EC09     INSTALLATION DONE',
#  'RISHIK U19EC157     INSTALLATION DONE',
#  'POOJA U19EC063     ASSIGNMENT1 IN PROGRESS',
#  'JANVI U19EE037     ASSIGNMENT1 IN PROGRESS',
#  'CHIRAG U19ME222     ASSIGNMENT2 IN PROGRESS',
#  'MUDIT U19EE039     ASSIGNMENT2 IN PROGRESS',
#  'MANISH U19ME114     ASSIGNMENT2 DONE',
#  'DHRUMIK U19ME224     ASSIGNMENT1 IN PROGRESS',
#  'KARANKUMAR U19ME002     INSTALLATION IN PROGRESS',
#  'OZA  U19EC0160     ASSIGNMENT2 IN PROGRESS',
#  'HEMIN U19EC045     INSTALLATION DONE',
#  'ANSHOO U19EC30     ASSIGNMENT2 IN PROGRESS',
#  'DHEERENDRA U19ME125     INSTALLATION DONE',
#  'TAKSH U19ME082     ASSIGNMENT1 IN PROGRESS',
#  'HEMIN U19EC045     ASSIGNMENT2 IN PROGRESS',
#  'BANSEEDHAR U19CS040     INSTALLATION IN PROGRESS',
#  'JAY U19CS044     ASSIGNMENT1 IN PROGRESS',
#  'HARSH U19EE031     INSTALLATION DONE', 
#  'PRACHI U19CH016     ASSIGNMENT1 IN PROGRESS', 
#  ]

# del students['U19EC30']
# del students['U19EC29']
# del students['U19EC09']
# del students['U19EC0160']

# students={}

# for i,ele in enumerate(names):
#     name_num,cur_status = ele.split('     ')
#     name,num = name_num.split()
#     indx = str(status.index(cur_status))
#     assert isinstance(indx, str)
#     students[f'{num}'] = [name,indx]

# student_list = [[x]+students[x] for x in students]
# student_list = sorted(student_list,key=lambda student_list: int(student_list[2]), reverse=True)
# student_list

# for i,ele in enumerate(status):
#     print(i,ele)

# for ele in student_list:
#     print(ele[:-1])

# student_info = generate_string(students)
# student_info

# for i,y in enumerate(status):
#     for j,ele in enumerate(names):
#         names[j] = names[j].replace(ele,str(i))

# inner_html

# message_id_outer_htlm

# #get all icons and try clicking on them till one brings edit in view
# icons = driver.find_elements_by_class_name('icons')
# print(icons)
# for ele in icons:
#     try:
#         ele.click()
#     except ElementNotInteractableException or StaleElementReferenceException:
#         continue

# driver.find_element_by_id(cke_contents_id_name)

# cke_contents = driver.find_element_by_css_selector(cke_wysiwyg_div)
# print(cke_contents.get_attribute('innerHTML'))
# # cke_contents.clear()

# '<div><div><div><div>test_abc</div><div>test3<br></div></div></div></div>'

# driver.execute_script(f"""arguments[0].setAttribute('innerHTML','<div><div><div><div>test_abc</div><div>test3<br></div></div></div></div>')""", cke_contents)

# cke_contents = driver.find_element_by_id(cke_contents_id_name)
# print(cke_contents.get_attribute('innerHTML'))

# '<div class="cke_wysiwyg_div cke_reset cke_enable_context_menu cke_editable cke_editable_themed cke_contents_ltr cke_show_borders" hidefocus="true" tabindex="0" spellcheck="true" role="textbox" aria-multiline="true" aria-label="Edit" data-tid="ckeditor-channelEdit" kbshort-ignore="24,25" contenteditable="true"><div><div><div><div>test</div></div></div></div></div>'

# driver.execute_script("""arguments[0].setAttribute('innerHTML','<div class="cke_wysiwyg_div cke_reset cke_enable_context_menu cke_editable cke_editable_themed cke_contents_ltr cke_show_borders" hidefocus="true" tabindex="0" spellcheck="true" role="textbox" aria-multiline="true" aria-label="Edit" data-tid="ckeditor-channelEdit" kbshort-ignore="24,25" contenteditable="true"><div><div><div><div>test</div></div></div></div></div>')""", cke_contents)

#cke_67_contents > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)

# driver.find_element_by_id(message_ids[2]).click()

# text = driver.find_element_by_css_selector("#cke_44_contents > div:nth-child(1)")

#cke_19_contents > div:nth-child(1) > div:nth-child(1)

#cke_44_contents > div:nth-child(1) > div:nth-child(1)

# .expanded > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)

# text.get_attribute("innerHTML")

# driver.executeScript("document.getElementById('//id of element').setAttribute('attr', '10')");

# script = "arguments[0].insertAdjacentHTML('afterbegin', arguments[1])"
# driver.execute_script(script, text)

# text.clear()

# script = "arguments[0].insertAdjacentHTML('beforebegin', arguments[1])"
# driver.execute_script(script, driver.find_elements_by_css_selector(br_tag_css_selector), st)

# driver.find_element_by_id('69d22375-a357-4bca-a45e-e1bda1cf7862').click()

# message_html = str(message_to_edit.get_attribute('innerHTML'))
# message_html

# driver.find_element_by_class_name('icons').click()

# /html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[9]/div/thread/div/div[2]/thread-body/div/div[2]/div/new-message/div/div[2]/form/div[4]/div[1]/div[3]/div/div/div[2]/div/div/div/div

# search_field = driver.find_element_by_xpath('//*[@id="searchInputField"]')
# search_field.send_keys(TEAM) 

# driver.find_elements_by_css_selector('img.profile-img')[0].click()

# # hover over Teams tab
# teams_tab = driver.find_element_by_id("app-bar-2a84919f-59d8-4441-a975-2a8c2643b741")
# hover = ActionChains(driver).move_to_element(teams_tab)
# hover.perform()

# bot_channel = driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[1]/div/left-rail/div/div/school-app-left-rail/single-team-channel-list/div/div/div/div[3]/ul/li[2]/a/div[1]/span").click()

# driver.get('https://teams.microsoft.com/_#/school/conversations/BOT?threadId=19:84124691e65d4ebabaee9d78e41ee04f@thread.tacv2&ctx=channel')

#messageBody

# bot_soup = bs(driver.page_source,'html.parser')

# bot_soup.findAll('div',attrs={'id','messageBody'})

# message_bodies[0].click()

# message_bodies[0].get_attribute('outerHTML')

# message_bodies[0].get_property('attributes')

#.a.attrs

# html = message_bodies[0].get_attribute('outerHTML')
# attrs = bs(html, 'html.parser')
# attrs

# hidden_buttons = driver.find_elements_by_css_selector("#messageBody")
# hidden_button[0].click()

# html.firefox.performance-mode-normal body.exp-density-view.hide-messages.power-bar-visible.initialized.loadingscreendone div#outer-shell.outer-shell div#wrapper.ts-main.no-right-rail.right-rail-ui-updates div#page-content-wrapper.ts-middle.ts-main-flex div.flex-fill middle-messages-stripe.flex-fill div.ts-middle-pane.flex-fill messages-header.messages-header.flex-fill div.ts-tab-content.flex-fill div.ts-embedded-container.flex-fill message-pane.content-child.flex-fill div.message-pane.flex-fill.with-chat-bubbles.new-typing-indicator div.ts-middle-messages-container.flex-fill div.ts-middle-messages-body.flex-fill div.ts-message-content.flex-fill message-list.flex-fill.message-list-common.smooth-highlight div.flex-fill virtual-repeat.ts-message-list.no-animation.simple-scrollbar div.list-wrap.list-wrap-v3.ts-message-list-container div.item-wrap.ts-message-list-item div#t1591546443115.clearfix thread.ts-expanded-message div.ts-message.acc-message-list-focusable div.conversation-common.conversation-start.conversation-not-collapsed thread-body div#m1591546443115.media.thread-body.acc-thread-focusable.has-hover-actions.keep-size-after-like.self div.ts-message-thread-body.align-item-left div#messageBody.message-body.message-body-width div.message-body-top-row.padded-content div.top-row-ext-container div.ts-msg-name.app-small-font

#m1591546443115 > div:nth-child(2) > div:nth-child(1)

# try:
#     hidden_button = driver.find_element_by_class_name("copy-paste-block")
# except NoSuchElementException :
#     try:
#         hidden_button = driver.find_elements_by_css_selector("#messageBody")
#     except:
#         hidden_button = driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[9]/div/thread/div/div[2]/thread-body/div/div[2]/div/div[1]/div/div[1]")

# hover = ActionChains(driver).move_to_element(hidden_button)
# hover.perform()

# div.ts-message-thread-body:nth-child(3) > div:nth-child(1)
# div.ts-message-thread-body:nth-child(2) > div:nth-child(1)
# div.ts-message-thread-body:nth-child(2) > div:nth-child(1)

# driver.find_elements_by_css_selector('svg.icons-more:nth-child(2)')[0].click()

# driver.find_elements_by_css_selector('li.message-action-overflow-menu-item:nth-child(2) > button:nth-child(1)')[0].click()

#cke_90_contents > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)

# driver.find_element_by_css_selector('#m1591546443115 > div:nth-child(2)').click()

#m1591546443115

# txt = driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[9]/div/thread/div/div[2]/thread-body/div/div[2]/div/new-message/div/div[2]/form/div[4]/div[1]/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div/div/div/div")

# txt.clear()

# driver.find_element_by_class_name('ts-popover-label').click()

#t1591523432403 > thread:nth-child(2) > div:nth-child(1)

# text_box = driver.find_element_by_css_selector('#cke_18_contents > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)')

# text_box = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[9]/div/thread/div/div[2]/thread-body/div/div[3]/div/new-message/div/div[2]/form/div[4]/div[1]/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div')

# text_box.clear()

# text_box.get_attribute('innerHtml')

# x_path = '/html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[9]/div/thread/div/div[2]/thread-body/div/div[3]/div/new-message/div/div[2]/form/div[4]/div[1]/div[3]/div/div/div[2]/div/div/div/div/div/div/div/div'

# txt = driver.find_elements_by_css_selector('#cke_100_contents > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)')

# txt[0]

# st = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Sed cursus ante dapibus diam. Sed nisi. Nulla quis sem at nibh elementum imperdiet. Duis sagittis ipsum. Praesent mauris. Fusce nec tellus sed augue semper porta. Mauris massa. Vestibulum lacinia arcu eget nulla. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Curabitur sodales ligula in libero. Sed dignissim lacinia nunc. 

# Curabitur tortor. Pellentesque nibh. Aenean quam. In scelerisque sem at dolor. Maecenas mattis. Sed convallis tristique sem. Proin ut ligula vel nunc egestas porttitor. Morbi lectus risus, iaculis vel, suscipit quis, luctus non, massa. Fusce ac turpis quis ligula lacinia aliquet. Mauris ipsum. Nulla metus metus, ullamcorper vel, tincidunt sed, euismod in, nibh. Quisque volutpat condimentum velit. 

# Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Nam nec ante. Sed lacinia, urna non tincidunt mattis, tortor neque adipiscing diam, a cursus ipsum ante quis turpis. Nulla facilisi. Ut fringilla. Suspendisse potenti. Nunc feugiat mi a tellus consequat imperdiet. Vestibulum sapien. Proin quam. Etiam ultrices. Suspendisse in justo eu magna luctus suscipit. Sed lectus. Integer euismod lacus luctus magna. Quisque cursus, metus vitae pharetra auctor, sem massa mattis sem, at interdum magna augue eget diam. 

# Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Morbi lacinia molestie dui. Praesent blandit dolor. Sed non quam. In vel mi sit amet augue congue elementum. Morbi in ipsum sit amet pede facilisis laoreet. Donec lacus nunc, viverra nec, blandit vel, egestas et, augue. Vestibulum tincidunt malesuada tellus. Ut ultrices ultrices enim. Curabitur sit amet mauris. Morbi in dui quis est pulvinar ullamcorper. Nulla facilisi. Integer lacinia sollicitudin massa. 

# Cras metus. Sed aliquet risus a tortor. Integer id quam. Morbi mi. Quisque nisl felis, venenatis tristique, dignissim in, ultrices sit amet, augue. Proin sodales libero eget ante. Nulla quam. Aenean laoreet. Vestibulum nisi lectus, commodo ac, facilisis ac, ultricies eu, pede. Ut orci risus, accumsan porttitor, cursus quis, aliquet eget, justo. Sed pretium blandit orci. Ut eu diam at pede suscipit sodales. Aenean lectus elit, fermentum non, convallis id, sagittis at, neque. 
# """

# br_tag_x_path = '/html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[9]/div/thread/div/div[2]/thread-body/div/div[2]/div/new-message/div/div[2]/form/div[4]/div[1]/div[3]/div/div/div[2]/div/div/div/div/br'
# br_tag_css_selector = '#cke_251_contents > div:nth-child(1) > div:nth-child(1) > br:nth-child(1)'

# /html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[5]/div/thread/div

# /html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[6]/div/thread/div

# /html/body/div[1]/div[2]/div[2]/div[1]/middle-messages-stripe/div/messages-header/div[2]/div/message-pane/div/div[1]/div/div/message-list/div/virtual-repeat/div/div[8]/div/thread/div

# soup = bs(page.content)



# cke_contents_id_name = re.findall(r'<div id="(cke_[0-9]*_contents)"',message_id_outer_htlm)
# cke_contents_id_name

# # cke_wysiwyg_div  = f"#{cke_contents_id_name} > div:nth-child(1)"
# cke_element = driver.find_element_by_css_selector(cke_wysiwyg_div)

# #find cke_element to dump payload into
# message_ids = re.findall(r'<div id="(m[0-9]*)"',str(driver.page_source))
# message_ids

# message_id_outer_htlm = str(driver.find_element_by_id(message_ids[-1]).get_attribute('innerHTML'))
# message_id_outer_htlm